In [25]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, clear_output
import ipywidgets as widgets
import json
import numpy as np
from PIL import Image

from datasets import load_dataset

ds = load_dataset("keremberke/plane-detection", name="full", split="train")


In [26]:
def plot_image_with_bbox(entry, clock_image_path="clock-without-hands-black-white.png"):
    # plane image
    img = entry['image']
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)
    
    objects = entry['objects']
    if objects['bbox']:
        bbox = objects['bbox'][0]  # [x, y, width, height]
        x, y, w, h = bbox
        
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        
        clock_img = Image.open(clock_image_path).convert("RGBA")
        clock_img = clock_img.resize((int(w), int(h)))
        clock_np = np.array(clock_img)
        
        # overlaying the clock image
        ax.imshow(clock_np, extent=[x, x+w, y+h, y], alpha=0.6)
    else:
        ax.text(0.5, 0.5, "No bounding box available", transform=ax.transAxes, 
                fontsize=20, ha='center', color='red')
    
    # showing the whole image
    ax.set_xlim(0, img.width)
    ax.set_ylim(img.height, 0)
    ax.set_title("Image with Bounding Box and Clock Overlay")
    plt.axis('off')
    plt.show()

In [27]:
%matplotlib inline
direction_widget = widgets.Dropdown(
    options=[str(i) for i in range(1, 13)],
    value='1',
    description='Direction:',
    style={'description_width': 'initial'}
)

contrail_widget = widgets.Checkbox(
    value=False,
    description='Contrail Visible',
    indent=False
)

plane_visible_widget = widgets.Checkbox(
    value=True,
    description='Plane Visible',
    indent=False
)

submit_button = widgets.Button(
    description="Submit Annotation",
    button_style='success'
)

annotation_out = widgets.Output()

annotations = {}

current_index = 0
max_annotations = 50


In [28]:
def next_image():
    global current_index, max_annotations
    plt.close('all')              # close open figures
    clear_output(wait=True)
    
    if current_index < max_annotations and current_index < len(ds['train']):
        entry = ds['train'][current_index]
        plot_image_with_bbox(entry)
        display(widgets.HBox([direction_widget, contrail_widget, plane_visible_widget, submit_button]))
        display(annotation_out)
        print(f"Image {current_index+1} of {max_annotations}")
    else:
        print("Annotation complete!")
        save_annotations()

def save_annotations():
    with open("annotations.json", "w") as f:
        json.dump(annotations, f, indent=4)
    print("Annotations saved to annotations.json")

def on_submit(change):
    global current_index, annotations
    direction = direction_widget.value
    contrail = contrail_widget.value
    plane_visible = plane_visible_widget.value
    
    image_id = ds['train'][current_index]['image_id']
    annotations[image_id] = {
        'direction': int(direction) if ds['train'][current_index]['objects']['bbox'] else None,
        'contrail': contrail,
        'plane_visible': plane_visible
    }
    
    with annotation_out:
        print(f"Annotated image_id {image_id} -> Direction: {direction}, Contrail: {contrail}, Plane Visible: {plane_visible}")
    
    # resetting widgets
    direction_widget.value = '1'
    contrail_widget.value = False
    plane_visible_widget.value = True
    
    current_index += 1
    next_image()

submit_button.on_click(on_submit)


In [29]:
next_image()

KeyError: "Column train not in the dataset. Current columns in the dataset: ['image_id', 'image', 'width', 'height', 'objects']"